In [1]:
import pandas as pd
import numpy as np
import video.df as df
from video import vid



In [2]:
path = 'videos/Red Spin.mp4'
vid = vid.Video.from_file(path)
vid_df = df.get_vid_df(path)

In [8]:
#vid.difference(1, 'lightness')

In [10]:
vid_df.head()

hue  lightness  saturation
frame y x                            
0     0 0    0        126         255
        1    0        126         255
        2    0        126         255
        3    0        126         255
        4    0        126         255

In [3]:
edf = df.get_exploration_df(path)

In [4]:
agg_df = df.get_aggregated_df(edf)

In [64]:
edf.head(2)

hue  lightness  saturation  masked_values  seconds
frame y x                                                    
0     0 0    0        126         255              0        0
        1    0        126         255              0        0

In [65]:
agg_df.head(2)

,lightness,hue,saturation,masked_values,light_diff,hue_diff,saturation_diff,mask_diff
frame,,,,,,,,
0,33.5124,0.1416,67.1892,0,NaN,NaN,NaN,NaN
1,32.5144,0.0004,65.4256,0,0.998,0.1412,1.7636,0.0


In [66]:
edf.shape, agg_df.shape

((750000, 5), (300, 8))

In [5]:
import os

# specify the directory path
directory_path = "videos1"

# get the list of filenames in the directory
filenames = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]

# print the list of filenames
print(filenames)


['hazard.01.mp4', 'hazard.02.mp4', 'safe.04.mp4', 'hazard.03.mp4', 'safe.01.mp4', 'hazard.07.mp4', 'hazard.06.mp4', 'data_list.pkl', 'safe.02.mp4', 'hazard.10.mp4', 'hazard.04.mp4', 'hazard.05.mp4', 'safe.03.mp4', 'hidden.05.mp4', 'hazard.08.mp4', 'hazard.09.mp4', 'hidden.04.mp4', 'hidden.06.mp4', 'hidden.03.mp4', 'hidden.02.mp4', 'data.pkl', 'hidden.01.mp4']


In [6]:
# get the list of filenames in the directory
filenames1 = os.listdir(directory_path)

# print the list of filenames
print(filenames1)

['hazard.01.mp4', 'hazard.02.mp4', 'safe.04.mp4', 'hazard.03.mp4', 'safe.01.mp4', 'hazard.07.mp4', 'hazard.06.mp4', 'data_list.pkl', 'safe.02.mp4', 'hazard.10.mp4', 'hazard.04.mp4', 'hazard.05.mp4', 'safe.03.mp4', 'hidden.05.mp4', 'hazard.08.mp4', 'hazard.09.mp4', 'hidden.04.mp4', 'hidden.06.mp4', 'hidden.03.mp4', 'hidden.02.mp4', 'data.pkl', 'hidden.01.mp4']


In [7]:
import video.reader as r
import cv2 as cv
from video.reader import VideoReader

In [8]:
vid, fps = r.VideoReader.get_vid(path, cv.COLOR_BGR2HLS)

In [30]:
vid.shape

(300, 50, 50, 3)

In [40]:
vid[0][0][1]

array([  0, 126, 255], dtype=uint8)

In [55]:
vid.reshape(-1,3).shape

(750000, 3)

In [72]:
lightness_per_frame = vid.reshape(-1, 3)[:, 1].reshape((-1, 50*50))

In [73]:
av_lightness_per_frame = np.mean(lightness_per_frame,axis=1)

In [74]:
av_lightness_per_frame.shape

(300,)

In [76]:
av_lightness_per_frame[:5]

array([33.5124, 32.5144, 31.4948, 32.4748, 33.5124])

In [78]:
agg_df.lightness[:5]

frame
0    33.5124
1    32.5144
2    31.4948
3    32.4748
4    33.5124
Name: lightness, dtype: float64

In [121]:
shifed_lightness = np.concatenate([np.zeros(1), av_lightness_per_frame[:-1]])

In [113]:
shifed_lightness[0] = np.nan

In [107]:
agg_df.lightness.shift(1)[:5]

frame
0        NaN
1    33.5124
2    32.5144
3    31.4948
4    32.4748
Name: lightness, dtype: float64

In [114]:
shifed_lightness[:5]

array([    nan, 33.5124, 32.5144, 31.4948, 32.4748])

In [122]:
diff_lightness = shifed_lightness - av_lightness_per_frame

In [123]:
diff_lightness[:5]

array([-33.5124,   0.998 ,   1.0196,  -0.98  ,  -1.0376])

In [118]:
agg_df.light_diff[:5]

frame
0       NaN
1    0.9980
2    1.0196
3   -0.9800
4   -1.0376
Name: light_diff, dtype: float64

In [119]:
vid.shape

(300, 50, 50, 3)

In [120]:
lightness_per_frame.shape

(300, 2500)

In [9]:
from typing import List, Union
from numpy.typing import ArrayLike

In [10]:
def get_lightness_difference(vid: Union[str, ArrayLike], fps: int = 30,
               conversion: int = cv.COLOR_BGR2HLS) -> np.array:
    '''
    returns an numpy array of difference in mean of lightness between frames.
    the length of the array = number of video frames - 1
    '''
    if isinstance(vid, str):
        vid, fps = VideoReader.get_vid(vid, conversion)
    frames = vid.shape[0]
    print(frames)
    height = vid.shape[1]
    width = vid.shape[2]
    # creates an numpy array with the lightness values of each frame
    # the shape of the array is (frames, height*width) f.e. (300, 2500)
    lightness_per_frame = vid.reshape(-1, 3)[:, 1].reshape((-1, height*width))
    # aggregate by mean
    av_lightness_per_frame = np.mean(lightness_per_frame,axis=1)
    # shift values by 1 position down
    shifed_lightness = np.concatenate([np.zeros(1), av_lightness_per_frame[:-1]])
    # set 1st value to NaN
    shifed_lightness[0] = np.nan
    # get the difference in lightness between frames
    diff_lightness = shifed_lightness - av_lightness_per_frame
    # return all values but NaN
    return diff_lightness[1:]

In [14]:
import re

In [138]:
re.search('(\w+)', filenames[0]).group()

'hazard'

In [12]:
directory = 'videos1/'

In [19]:
for f in filenames:
    if f.endswith('.pkl'):
        filenames.remove(f)

In [20]:
filenames

['hazard.01.mp4',
 'hazard.02.mp4',
 'safe.04.mp4',
 'hazard.03.mp4',
 'safe.01.mp4',
 'hazard.07.mp4',
 'hazard.06.mp4',
 'safe.02.mp4',
 'hazard.10.mp4',
 'hazard.04.mp4',
 'hazard.05.mp4',
 'safe.03.mp4',
 'hidden.05.mp4',
 'hazard.08.mp4',
 'hazard.09.mp4',
 'hidden.04.mp4',
 'hidden.06.mp4',
 'hidden.03.mp4',
 'hidden.02.mp4',
 'hidden.01.mp4']

In [101]:
def get_file_names(directory:str):
    '''
    returns the list of video files in the directory
    '''
    # get the list of filenames in the directory
    filenames = \
        [f for f in os.listdir(directory) 
        if (os.path.isfile(os.path.join(directory, f)) and f.endswith('.mp4'))]
    return filenames

In [ ]:
def get_sequence(directory: str):
    '''
    '''
    pickle_file = 'data.pkl'
    pickle_path = directory + '/' + pickle_file
    if os.path.isfile(pickle_path):
        with open(pickle_path, "rb") as fp:
        x = pickle.load(fp)
    filenames = get_file_names(directory)
    data = {'Sequence':[], 'ClassName':[]}
    for f in filenames:
        path = directory + '/' + f
        sq = get_lightness_difference(path)
        class_name = re.search('(\w+)', f).group()
        data['Sequence'].append(sq)
        data['ClassName'].append(class_name)

    
    with open("videos1/data.pkl", "wb") as outfile1:
    pickle.dump(data, outfile1)
        

In [21]:
data_list = []
data = {'Number':[], 'Sequence':[], 'ClassName':[]}
for i, file_name in enumerate(filenames):
    path = directory + file_name
    sq = get_lightness_difference(path)
    #print(len(sq))
    class_name = re.search('(\w+)', file_name).group()
    data_list.append({'Number': i, 'Sequence': sq, 'ClassName': class_name}) 
    data['Number'].append(i)
    data['Sequence'].append(sq)
    data['ClassName'].append(class_name)
    

3927
785
4561
300
768
7346
150
2304
1521
150
6314
297
3933
300
2375
841
1159
407
888
1290


In [179]:
path = 'videos1/safe.04.mp4'
df1 = df.get_exploration_df(path)

In [180]:
df1.head(1)

,,,hue,lightness,saturation,masked_values,seconds
frame,y,x,,,,,
0,0,0,60,99,118,0,0


In [181]:
agg_df1 = df.get_aggregated_df(df1)

In [182]:
agg_df1.tail(2)

,lightness,hue,saturation,masked_values,light_diff,hue_diff,saturation_diff,mask_diff
frame,,,,,,,,
4559,251.0644,4.17,8.0192,2455,0.0000,0.00,0.0000,0.0
4560,255.0000,0.00,0.0000,2500,-3.9356,4.17,8.0192,-45.0


In [22]:
import json
import pickle

In [23]:
with open("videos1/data_list.pkl", "wb") as outfile:
    pickle.dump(data_list, outfile)
with open("videos1/data.pkl", "wb") as outfile1:
    pickle.dump(data, outfile1)

In [153]:
with open("videos1/data.pkl", "rb") as f:
    x = pickle.load(f)

In [36]:

# Load the vectors into a list
vectors = data['Sequence']

# Determine the maximum length of the vectors
max_len = max(len(v) for v in vectors)

# Pad the vectors
padded_vectors = []
for v in vectors:
    padded_vector = np.pad(v, (0, max_len - len(v)), 'reflect')
    padded_vectors.append(padded_vector)

# Create a mask
mask = []
for v in padded_vectors:
    mask.append([1] * len(v) + [0] * (max_len - len(v)))

# Convert the lists to numpy arrays
padded_vectors = np.array(padded_vectors)
mask = np.array(mask)

data['Sequence'] = padded_vectors
# Label the vectors
#labels = np.array([0, 1, 0])


In [49]:
type(data['Sequence'])

numpy.ndarray

In [88]:

data_for_labeling = np.array(data['Sequence'])

# create a corresponding label array 
labels = np.array(data['ClassName'])

from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
# make sure that the label array is of the same data type as the numpy array
le = LabelEncoder()
num_labels = le.fit_transform(labels)
labels = num_labels.astype(data_for_labeling.dtype)

labels = np_utils.to_categorical(labels)



In [38]:
data_for_labeling.shape

(20, 7345)

In [39]:
labels

array(['hazard', 'hazard', 'safe', 'hazard', 'safe', 'hazard', 'hazard',
       'safe', 'hazard', 'hazard', 'hazard', 'safe', 'hidden', 'hazard',
       'hazard', 'hidden', 'hidden', 'hidden', 'hidden', 'hidden'],
      dtype='<U6')

In [41]:
labels

array([0., 0., 2., 0., 2., 0., 0., 2., 0., 0., 0., 2., 1., 0., 0., 1., 1.,
       1., 1., 1.])

0 - hazard

1 - hidden

2 - safe

In [185]:
path = 'videos1/safe.04.mp4'
vid, fps = VideoReader.get_vid(path, cv.COLOR_BGR2HLS)

In [189]:
    frames = vid.shape[0]
    height = vid.shape[1]
    width = vid.shape[2]

In [190]:
lightness_per_frame = vid.reshape(-1, 3)[:, 1].reshape((-1, height*width))

In [191]:
lightness_per_frame.shape

(4561, 2500)

In [192]:
shifed_lightness = np.concatenate([np.zeros(1), av_lightness_per_frame[:-1]])

In [193]:
shifed_lightness.shape # found an error

(300,)

In [44]:
import tensorflow as tf

2023-04-10 11:59:01.192688: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [58]:
from sklearn.preprocessing import StandardScaler

In [59]:
sc = StandardScaler()
data_for_labeling = sc.fit_transform(data_for_labeling)

In [89]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=5000, activation='relu'))
ann.add(tf.keras.layers.Dense(units=3, activation='softmax'))

In [90]:
ann.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [91]:
ann.fit(data_for_labeling, labels,  batch_size=64, epochs=50)

Epoch 1/50
1/1 [==============================] - 1s 1s/step - loss: 8.7662 - accuracy: 0.5000
Epoch 2/50
1/1 [==============================] - 0s 413ms/step - loss: 38.1417 - accuracy: 0.8000
Epoch 3/50
1/1 [==============================] - 0s 440ms/step - loss: 5.9605e-09 - accuracy: 1.0000
Epoch 4/50
1/1 [==============================] - 0s 414ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/50
1/1 [==============================] - 0s 422ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/50
1/1 [==============================] - 0s 426ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/50
1/1 [==============================] - 1s 502ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/50
1/1 [==============================] - 0s 446ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/50
1/1 [==============================] - 0s 406ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 10/50
1/1 [==============================] - 0s 397ms/step - loss: 0.0000e+00 - accura

In [82]:
ann.predict(data_for_labeling)

1/1 [==============================] - 0s 106ms/step


array([[0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [0.99999994],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ]], dtype=float32)

In [92]:
labels

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

In [93]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold